# **Machine Learning Lab 4EII - IA course**
***Lab 5 - Transfer Leearning with Keras***

## 1- **Introduction**

This lab aims at introducing two important concepts of **transfer learning** in Machine Learning (ML) which are **features extraction** and **fine tuning**.  

In this lab you will learn to:
* Perform features extraction and classification from a pre-trained deep neural network
* Perform fine tuning of a pre-trained deep neural network
* Apply these two concepts to food classification problem
	

## 2- **Definition of transfer learning**

Transfer learning is the process of:

1.   Taking a network pre-trained on a dataset
2.   Use this pre-trained network to recognize image/object categories it was not trained on


 

> In which situations we use **transfer learning** ?

The idea behind transfer learning is to take a network trained on a huge dataset like ImageNet ( [ImageNet](http://www.image-net.org/) is a dataset of more then a million of images with 1000 outputs/classes) and use it for another classification of regression problem. 

In general, there are two types of transfer learning in the context of deep learning:

1. Transfer learning via feature extraction
2. Transfer learning via fine-tuning

In this Lab, you will study thses two concepts of transfer learning based on VGG16 Network trained on ImageNet dataset for image classification. The architecture of VGG16 is illustrated in this Figure. Please refer to this figure in this Lab.  

![Architecture of VGG16](https://drive.google.com/uc?id=1cfhg1AmBhj6K0-fdvdKQzQrZqOMgmz27)






## 3- **Features extraction**



In this section you will see how to use features extraction from a pre-trained deep neural network trained on ImageNet dataset to perform classification of images in two classes: Food or Non-food.

## 3.a - Dataset preparation

The first step consists in dataset preparation. We use here a [Food-5K dataset](https://www.epfl.ch/labs/mmspg/downloads/food-image-datasets/) provided by MMSPG - EPFL. This dataset includes 5K images (different from ImageNet images) where 1K images are used for validation, 1K for testing and 3K for training. The images belong to two classes:

1 - Food (2500 images)

2 - Non-food (2500 images)  

Now you have to configure your working directory and download the dataset: 

1 - Create a folder called TP5-IA 

2 - Upload the provided dataset in your working directory. 

In [0]:
import os
dirPath = '/content/TP5-IA/'
if not os.path.exists(dirPath):
  os.makedirs(dirPath)
os.chdir(dirPath)

In [0]:
base_path = 'Food-5K-dataset' 

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1tigIJiCg7nDsoOEg97ark_yWQEakAKKU' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1tigIJiCg7nDsoOEg97ark_yWQEakAKKU" -O dataset-20200503T154145Z-001.zip && rm -rf /tmp/cookies.txt
!unzip dataset-20200503T154145Z-001.zip 
%rm 'dataset-20200503T154145Z-001.zip'
os.rename('dataset', base_path) 

You can browse the downloaded directory '/content/Food-5K-dataset' to see how the files and images are organized in the dataset. Keras provides some powerful functions to efficiently manage a dataset organized in this way. 

You can use this method to display some images of this dataset


In [0]:
def displayImage (input_image):
  import cv2
  from google.colab.patches import cv2_imshow

  image = cv2.imread(input_image)
  output = image.copy()
  cv2_imshow(output)
  cv2.waitKey(0)


In [0]:
image = 'Food-5K-dataset/training/food/1_119.jpg'
#image = 'Food-5K-dataset/training/non_food/0_100.jpg'
displayImage(image)


The following Config class initialises the paths of different sets.

In [0]:
# Initialisation class 
from imutils import paths
class Config: 
# initialize the path to the *original* input directory of images
  def __init__(self, base_path):
    
#    self.ORIG_INPUT_DATASET = "Food-5K"

# initialize the base path to the *new* directory that will contain
# our images after computing the training and testing split
    self.BASE_PATH = base_path;

# define the names of the training, testing, and validation
# directories
    self.TRAIN = "training"
    self.TEST = "evaluation"
    self.VAL = "validation"

# initialize the list of class label names
    self.CLASSES = [];
    
    if(base_path == 'Food-5K-dataset'):
      self.CLASSES = ["non_food", "food"]
    
    if(base_path == 'Food-11-dataset'):
      self.CLASSES = ["Bread", "Dairy product", "Dessert", "Egg", "Fried food", 
                      "Meat", "Noodles/Pasta", "Rice", "Seafood", "Soup", 
                      "Vegetable/Fruit"]

# set the batch size
    self.BATCH_SIZE = 32

# initialize the label encoder file path and the output directory to
# where the extracted features (in CSV file format) will be stored
    self.LE_PATH = os.path.sep.join(["output", "le.cpickle"])
    self.BASE_CSV_PATH = "output"

# set the path to the serialized model after training
    self.MODEL_PATH = os.path.sep.join(["output", "model.cpickle"])

# define the path to the output training history plots
    self.UNFROZEN_PLOT_PATH = os.path.sep.join(["output", "unfrozen.png"])
    self.WARMUP_PLOT_PATH = os.path.sep.join(["output", "warmup.png"])

    self.trainPath = os.path.sep.join([self.BASE_PATH, self.TRAIN])
    self.valPath = os.path.sep.join([self.BASE_PATH, self.VAL])
    self.testPath = os.path.sep.join([self.BASE_PATH, self.TEST])
    self.totalTrain = len(list(paths.list_images(self.trainPath)))
    self.totalVal = len(list(paths.list_images(self.valPath)))
    self.totalTest = len(list(paths.list_images(self.testPath)))
  
  def print_info(self):
    
# determine the total number of image paths in training, validation,
# and testing directories
    
    print('Size of training set : ',  self.totalTrain)
    print('Size of validation set : ',  self.totalVal)
    print('Size of testing set : ',  self.totalTest)


Create an instance of the *Config* class for Food-5K-dataset and print some information with *print_info* method:


In [0]:
base_path = 'Food-5K-dataset' 
#TO DO 

## 3.b - Features extraction

The idea behind features extraction is to use a pre-trained network to extracted features that will be used as input of a simple machine learning algorithms like SVM, Random Forest or LogisticRegression. In fact, features extracted from a deep neural network trained on a huge dataset can be useful for many other classification problems. As pre-trained network, we can use here several models trained on ImageNet data with million of images and 1000 classes such as VGG16, ResNet, Inception, Xception, etc. 

In this Lab, you will use VGG16, but you can also test other networks. The weights of the pre-trained VGG16 network can be downloaded from Keras library. You can notice that we are interested by features from VGG16 and not the output classes. So, we download the VGG16 Network without the last classification output layer (without dense layers in green in the VGG-16 figure). You can print the network configuration.  

In [0]:
from tensorflow.keras.applications import VGG16
modelVGG16 = VGG16(weights="imagenet", include_top=False) # include_top = False: exclude the last dense layers 
print(modelVGG16.summary())

Now, you have to build a new class called *FeaturesExtraction* that extracts features using the VGG16 model on Food-5K dataset images. The extracted features and the corresponding labels will be saved in lists: (trainX, trainY), (testX, testY), (valX, valY) containing features and labels of the three sets training, testing and validation of the dataset. You will forward propagate the dataset images in the VGG16 network to extract features.  

In [0]:
# USAGE
# python extract_features.py

# import the necessary packages
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img


import numpy as np
import pickle
import random


class FeaturesExtraction:
  
  def __init__(self, model, config):
    
    self.model = model; 
    self.config = config;

    self.reset();

  
  def reset (self):
    self.trainX = []
    self.trainY = []
    
    self.testX = []
    self.testY = []    
    
    self.valX = []
    self.valY = []
    self.le = None
    
  def extractFeatures(self):
    self.reset();   
    for (split, x, y) in ((self.config.TRAIN, self.trainX, self.trainY), (self.config.TEST,  self.testX, self.testY), (self.config.VAL, self.valX, self.valY)):
      print("[INFO] processing '{} split'...".format(split))
      p = os.path.sep.join([self.config.BASE_PATH, split])
      imagePaths = list(paths.list_images(p))
      print(p)
      

	    # randomly shuffle the image paths and then extract the class
	    # labels from the file paths
      random.shuffle(imagePaths)
      labels = [p.split(os.path.sep)[-2] for p in imagePaths]
      if self.le is None:
        self.le = LabelEncoder()
        self.le.fit(labels)

      for (b, i) in enumerate(range(0, len(imagePaths), self.config.BATCH_SIZE)):
		    # extract the batch of images and labels, then initialize the
		    # list of actual images that will be passed through the network
		    # for feature extraction
        print("[INFO] processing batch {}/{}".format(b + 1, int(np.ceil(len(imagePaths) / float(self.config.BATCH_SIZE)))))
        batchPaths = imagePaths[i:i + self.config.BATCH_SIZE]
        batchLabels = self.le.transform(labels[i:i + self.config.BATCH_SIZE])
        batchImages = []
        
		    # loop over the images and labels in the current batch
        for imagePath in batchPaths:
			    # load the input image using the Keras helper utility
			    # while ensuring the image is resized to 224x224 pixels
          image = load_img(imagePath, target_size=(224, 224))
          image = img_to_array(image)

			    # preprocess the image by (1) expanding the dimensions and
			    # (2) subtracting the mean RGB pixel intensity from the
			    # ImageNet dataset
          image = np.expand_dims(image, axis=0)
          image = imagenet_utils.preprocess_input(image)

			    # add the image to the batch
        
          batchImages.append(image)

		      # pass the images through the network and use the outputs as
		      # our actual features, then reshape the features into a
		      # flattened volume
        batchImages = np.vstack(batchImages)
        
        #TO DO 
        
        #Prediction batchImages

        #reshape the output from (see the output size of VGG-16) to 1D vector 

        
        for k in range(0, len(batchLabels)):  
          x.append(features[k])
          y.append(batchLabels[k])


  
 

Create an instance of the FeaturesExtraction.

In [0]:
#TO DO

Call the features extraction class. 

In [0]:
#TO DO 

## 3.c - Simple Machine Learning model

In this section you will create a simple SVM machine learning model then train and test it with features and labels of Food-5K images extracted from the VGG16 Network. You can use standard ML algorithms you have seen in Lab 1 : Random Forest, SVM (linear kernal) or LogisticRegression (solver='lbfgs'). SVM could be long for training.

In [0]:
from sklearn.linear_model import LogisticRegression
#TO DO

Perform evaluation of the model

In [0]:
from sklearn.metrics import classification_report
#TO DO

print(classification_report(self.testY, preds, target_names=self.le.classes_))

## 4- **Fine Tuning**

In this section you will perform fine tuning of a pre-trained model on ImageNet to solve a more challenging classification problem of Food-11 dataset. 



## 4.a - Dataset preparation

The [Food-11](https://www.epfl.ch/labs/mmspg/downloads/food-image-datasets/) dataset consists of 16643 images partitioned into three sets: training, testing and validation. These images belong to 11 major food categories: 

1 - Bread (1724 images)

2 - Dairy product (721 images)

3 - Dessert (2,500 images)

4 - Egg (1,648 images)

5 - Fried food (1,461images)

6 - Meat (2,206 images)

7 - Noodles/pasta (734 images)

8 - Rice (472 images)

9 - Seafood (1,505 images)

10 - Soup (2,500 images)

11 - Vegetable/fruit (1,172 images)

You need first to run the following blocks to create the configuration class of Food-11 dataset and download the dataset on your working directory.

In [0]:
dirPath = '/content/TP5-IA/'
if not os.path.exists(dirPath):
  os.makedirs(dirPath)
os.chdir(dirPath)

In [0]:
base_path = 'Food-11-dataset' 

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-lt3_fgg5kcnxcCTPYlPdGup95Aa9iCF' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-lt3_fgg5kcnxcCTPYlPdGup95Aa9iCF" -O dataset.zip && rm -rf /tmp/cookies.txt
!unzip dataset.zip 
%rm 'dataset.zip'
%rm -r '__MACOSX'
os.rename('dataset', base_path)



Create an instance of the Config class for Food-11-dataset:

In [0]:
base_path = 'Food-11-dataset' 
#TO DO

You can browse the downloaded directory '/content/Food-51-dataset' to see how the files and images are organized in the dataset. You can also display some samples of the dataset.

In [0]:
image = 'Food-11-dataset/training/Meat/5_0.jpg'
#image = 'Food-11-dataset/training/Soup/9_10.jpg'
#image = 'Food-11-dataset/training/Dessert/2_1.jpg'
displayImage(image)

In this section, you will create data generator for train, validation and test sets. For the training set, the generator will perform some data augmentation such as rotation, zoom, etc. This data augmentation is important for better training and the network will be more robust and accurate regarding those operations.

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

# initialize the training generator
trainGen = trainAug.flow_from_directory(
	configFood11.trainPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=True,
	batch_size=configFood11.BATCH_SIZE)

# initialize the validation generator
valGen = valAug.flow_from_directory(
	configFood11.valPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=configFood11.BATCH_SIZE)

# initialize the testing generator
testGen = valAug.flow_from_directory(
	configFood11.testPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=configFood11.BATCH_SIZE)


## 4.b - Fine-tuning classification model 

In this section you will build the classification network based on VGG16. The idea is to use the head of VGG16 with additional layers to adapt the output to this problem for food classification. You should add one Flatten layer, one dense layer of size 512 (relu), dropout layer (0.5) and the output layer with softmax activation function.  

The concept of fine tuning consists in three mains steps:

1 - Build the model based on the pre-trained network (here VGG16) as the head of the network and add fully connected layers to adpt the network to our classification problem. 

2 - Freeze the head layers from VGG16 and train the model with updating only new added layers (over 50 epochs). 

3 - Fine tune the model with updating the four last layers of VGG-16 and new dense layers on few epochs (20). 

Let'us perform these three steps

**1 - Build the model**

In [0]:
from tensorflow.keras.layers import Input, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16

# load the VGG16 network, ensuring the head FC layer sets are left
# off
baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model

headModel = baseModel.output

# TO DO

model = Model(inputs=baseModel.input, outputs=headModel)
print(model.summary())

**2 - Freeze the head layers from VGG and train the model**


This code enable to freeze the layers of VGG16 network. So, you will perform training only on the new layers.

In [0]:
for layer in baseModel.layers:
	layer.trainable = False

for layer in baseModel.layers:
	print("{}: {}".format(layer, layer.trainable))

Train the model 

In [0]:
from tensorflow.keras.optimizers import SGD

# compile our model (this needs to be done after our setting our
# layers to being non-trainable
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit_generator(
	trainGen,
	steps_per_epoch=config.totalTrain // config.BATCH_SIZE,
	validation_data=valGen,
	validation_steps=config.totalVal // config.BATCH_SIZE,
	epochs=50)


If it is too long to do the training, you can upload the provided weights. 

In [0]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-BHSwPB-cPty7K7BeQA_7-PTc_IXMybG' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-BHSwPB-cPty7K7BeQA_7-PTc_IXMybG" -O model_train.h5 && rm -rf /tmp/cookies.txt 

model.load_weights("model_train.h5")
print("Weights loaded from disk")

In [0]:
# reset the testing generator and evaluate the network after
# fine-tuning just the network head
from sklearn.metrics import classification_report
print("[INFO] evaluating after fine-tuning network head...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
	steps=(config.totalTest // config.BATCH_SIZE) + 1)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))

**3 - Fine tune the model**

Unfreeze the last four layers of VGG-16 network (Layers Conv 5-1, Conv 5-2, Conv 5-3 and pooling in Figure 1)

In [0]:
# now that the head FC layers have been trained/initialized, lets
# unfreeze the final set of CONV layers and make them trainable

#TO DO 

# loop over the layers in the model and show which ones are trainable
# or not
for layer in baseModel.layers:
	print("{}: {}".format(layer, layer.trainable))

Perform fine tuning of the network 

In [0]:
# reset our data generators
trainGen.reset()
valGen.reset()


# for the changes to the model to take affect we need to recompile
# the model, this time using SGD with a *very* small learning rate
print("[INFO] re-compiling model...")
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the model again, this time fine-tuning *both* the final set
# of CONV layers along with our set of FC layers
H = model.fit_generator(
	trainGen,
	steps_per_epoch=config.totalTrain // config.BATCH_SIZE,
	validation_data=valGen,
	validation_steps=config.totalVal // config.BATCH_SIZE,
	epochs=20)

If it is too long to do the training you can upload the saved weights after this fine-tuning. 

In [0]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-D-Y_ds9284h7i5J1892JLX30V_sIWaN' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-D-Y_ds9284h7i5J1892JLX30V_sIWaN" -O model_fine_tune.h5 && rm -rf /tmp/cookies.txt

model.load_weights("model_fine_tune.h5")
print("Weights loaded from disk")



Test the network 

In [0]:
# reset the testing generator and then use our trained model to
# make predictions on the data
print("[INFO] evaluating after fine-tuning network...")

testGen.reset()
predIdxs = model.predict_generator(testGen,
	steps=(config.totalTest // config.BATCH_SIZE) + 1)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))

# serialize the model to disk
if not os.path.exists(config.MODEL_PATH):
  os.makedirs(config.MODEL_PATH)

print("[INFO] serializing network...")
model.save(config.MODEL_PATH)

Load an image from the dataset and predict it:

In [0]:
def predict (input_image, config):
  from tensorflow.keras.models import load_model
  import numpy as np
  import imutils
  import cv2
  from google.colab.patches import cv2_imshow

  image = cv2.imread(input_image)
  output = image.copy()
  output = imutils.resize(output, width=400)
# our model was trained on RGB ordered images but OpenCV represents
# images in BGR order, so swap the channels, and then resize to
# 224x224 (the input dimensions for VGG16)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image, (224, 224))
# convert the image to a floating point data type and perform mean
# subtraction
  image = image.astype("float32")
  mean = np.array([123.68, 116.779, 103.939][::-1], dtype="float32")
  image -= mean


# load the trained model from disk
  print("[INFO] loading model...")
  model = load_model(config.MODEL_PATH)
# pass the image through the network to obtain our predictions
  preds = model.predict(np.expand_dims(image, axis=0))[0]
  i = np.argmax(preds)
  label = config.CLASSES[i]
# draw the prediction on the output image
  text = "{}: {:.2f}%".format(label, preds[i] * 100)
  cv2.putText(output, text, (3, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
	  (0, 255, 0), 2)
# show the output image
  cv2_imshow(output)
  cv2.waitKey(0)

def predict (input_image, config):
  from tensorflow.keras.models import load_model
  import numpy as np
  import imutils
  import cv2
  from google.colab.patches import cv2_imshow

  image = cv2.imread(input_image)
  output = image.copy()
  output = imutils.resize(output, width=400)
# our model was trained on RGB ordered images but OpenCV represents
# images in BGR order, so swap the channels, and then resize to
# 224x224 (the input dimensions for VGG16)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = cv2.resize(image, (224, 224))
# convert the image to a floating point data type and perform mean
# subtraction
  image = image.astype("float32")
  mean = np.array([123.68, 116.779, 103.939][::-1], dtype="float32")
  image -= mean


# load the trained model from disk
  print("[INFO] loading model...")
  model = load_model(config.MODEL_PATH)
# pass the image through the network to obtain our predictions
  preds = model.predict(np.expand_dims(image, axis=0))[0]
  i = np.argmax(preds)
  label = config.CLASSES[i]
# draw the prediction on the output image
  text = "{}: {:.2f}%".format(label, preds[i] * 100)
  cv2.putText(output, text, (3, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
	  (0, 255, 0), 2)
# show the output image
  cv2_imshow(output)
  cv2.waitKey(0)


You can predict the label of an image

In [0]:
image = 'Food-11-dataset/evaluation/Meat/5_1.jpg'
predict(image, config)